# Proof of concept: Calculate sun angle
Shadow length and time of day have been collected from http://centrodedescargas.cnig.es. Check
whether these can calculate a hub height that approximately matches the identified hub heights.

__Conclusions:__
1. The aerial photos have a timestamp in UTC
2. The quality of initial spanish metadata is awful (3 out of 7 are a match)
3. 5m is a reasonable accuracy goal (weak evidence, based on three matches)

In [ ]:
import os
from datetime import datetime

import dotenv
import numpy as np
import pandas as pd
from skyfield import almanac, api

dotenv.load_dotenv('../.env')
dotenv.load_dotenv('../.env.secret')

sites = pd.read_csv('../data/poc_measurements.csv')

In [ ]:
sites = sites.assign(
    date_string = lambda x: x.date + ' ' + x.hora + os.environ.get('utc_offset'),
    datetime_utc = lambda x: pd.to_datetime(x.date_string, dayfirst=True, utc=True)
)
sites

In [ ]:
ephemeris = api.load('de421.bsp')
earth, sun = ephemeris['earth'], ephemeris['sun']
test_sites = sites[~sites.hora.isna()].drop_duplicates(subset='site', keep='last')
for i, site in test_sites.iterrows():
    observer = earth + api.wgs84.latlon(latitude_degrees=site.latitude, longitude_degrees=site.longitude)
    time = api.load.timescale().from_datetime(site.datetime_utc)
    altitude, _, _  = observer.at(time).observe(sun).apparent().altaz()
    test_sites.loc[i, ['altitude_degrees', 'altitude_radians']]= altitude.degrees, altitude.radians
test_sites = test_sites.assign(
    estimated_hub_height=lambda x: np.tan(x.altitude_radians) * x.shadow_length
)
test_sites[[
    'site', 'latitude', 'longitude', 'hub_height', 'shadow_length', 'altitude_degrees', 'estimated_hub_height'
]].round(2)

In [ ]:
# When is the solar noon in Adrano?
turbine = api.wgs84.latlon(site.latitude, site.longitude)
t1 = api.load.timescale().utc(2019, 7, 14)
t2 = api.load.timescale().utc(2019, 7, 15)
f = almanac.meridian_transits(ephemeris, sun, turbine)
times, events = almanac.find_discrete(t1, t2, f)
times[1].tt_calendar()

In [ ]:
# How does the altitude angle for a day in June in Adrano?
angle_list = []
observer = earth + api.wgs84.latlon(latitude_degrees=site.latitude, longitude_degrees=site.longitude)
site = sites.iloc[0]
for hour in range(24):
    time = api.load.timescale().from_datetime(site.datetime_utc.replace(hour=hour))
    altitude, _, _  = observer.at(time).observe(sun).apparent().altaz()
    angle_list.append({'hour': hour, 'degrees': altitude.degrees, 'radians': altitude.radians})
pd.DataFrame(angle_list)

In [ ]:
# Which utc_offset minimises the errors in the estimated hub height?
result_list = []
for _, site in test_sites.iterrows():
    observer = earth + api.wgs84.latlon(latitude_degrees=site.latitude, longitude_degrees=site.longitude)
    site_result_dict = {}
    for utc_offset in range(3):
        date_string = datetime.strptime(
            f'{site.date} {site.hora} +{utc_offset:02}00', '%d/%m/%Y %H:%M:%S %z')
        time = api.load.timescale().from_datetime(date_string)
        altitude, _, _  = observer.at(time).observe(sun).apparent().altaz()
        estimated_hub_height = round(np.tan(altitude.radians) * site.shadow_length, 1)
        site_result_dict[f'estimate_{utc_offset}'] = estimated_hub_height

    result_list.append(site_result_dict)

results = pd.concat([
    test_sites[['site', 'hub_height', 'shadow_length']],
    pd.DataFrame(result_list, index = test_sites.index)]
    , axis=1
)
results['error_0'] = results.estimate_0 - results.hub_height
results

In [ ]:
summary_list = []
for utc_offset in range(3):
    errors = results[f'estimate_{utc_offset}'] - results.hub_height
    summary_list.append({
        'utc_offset': utc_offset,
        'within_5m': errors.between(-5, 5).sum(),
        'within_10m': errors.between(-10, 10).sum(),
        'within_20m': errors.between(-20, 20).sum()
    })
pd.DataFrame(summary_list)